In [81]:

import numpy as np
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import NonlinearConstraint
from scipy.optimize import LinearConstraint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cvxopt.modeling import op, variable, dot
from cvxopt import matrix 
from cvxopt import solvers
import time
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel
from copy import copy
import itertools
from tqdm import tqdm_notebook

In [82]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels



X_all_labels, y_all_labels = load_mnist('C:/Users/RE-Giorgio/Documents/OptimusPrime/Data', kind='train')


indexLabel2 = np.where((y_all_labels==2))
xLabel2 =  X_all_labels[indexLabel2][:1000,:].astype('float64') 
yLabel2 = y_all_labels[indexLabel2][:1000].astype('float64') 

indexLabel4 = np.where((y_all_labels==4))
xLabel4 =  X_all_labels[indexLabel4][:1000,:].astype('float64') 
yLabel4 = y_all_labels[indexLabel4][:1000].astype('float64') 

indexLabel6 = np.where((y_all_labels==6))
xLabel6 =  X_all_labels[indexLabel6][:100,:].astype('float64') 
yLabel6 = y_all_labels[indexLabel6][:100].astype('float64') 

yLabel2[:] = +1
yLabel4[:] = -1

X = np.concatenate([xLabel2, xLabel4])
y = np.concatenate([yLabel2, yLabel4])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.2, random_state=1696995) 

scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [85]:
class Svm_dcmp:
    
    def __init__(self, X, y, gamma, C, kernel, q):
        
        self.X = X
        self.y = y
        self.alpha = np.zeros((X.shape[0]))
        self.b = 0
        self.C = C
        self.gamma = gamma
        self.kernel = kernel
        self.q = q
        self.grad = - np.ones(X.shape[0])
        
    def predict(self,X):
        
        if self.kernel == "gauss":
            z = (self.alpha*self.y) @ self.kernel_gauss(self.X, X) + self.b
        if self.kernel == "poly":
            z = (self.alpha*self.y) @ self.kernel_poly(self.X, X) + self.b
        a = np.sign(z)    
        return a
    
    def kernel_gauss(self, X1, X2):
        return rbf_kernel(X1,X2, gamma = self.gamma)
    
    def kernel_poly(self, X1, X2):
        return polynomial_kernel(X1,X2, gamma = self.gamma)

    def get_working_set(self, alpha, K):
        
        # box constraints
        y = self.y.ravel(); C = self.C; q = self.q
        R = np.where((alpha < 1e-5) & (y == +1) | (alpha > C-1e-5) & (y == -1) | (alpha > 1e-5) & (alpha < C-1e-5))[0]
        S = np.where((alpha < 1e-5) & (y == -1) | (alpha > C-1e-5) & (y == +1) | (alpha > 1e-5) & (alpha < C-1e-5))[0]
        
        # negative gradient divided by y
        Q = np.outer(y,y) * K 
        grad = Q @ alpha - 1
        grady = - grad/y
        
        indexed = np.vstack((grady, np.arange(len(grady)).astype("int"))).T
        grady_R = indexed[R, :]
        I = list(np.array(sorted(indexed[R], key = lambda i: - i[0]))[:, 1].T.astype("int"))[:q//2]
        
        grady_S = indexed[S, :]
        J = list(np.array(sorted(indexed[S], key = lambda i: i[0]))[:, 1].T.astype("int"))[:q//2]
        W = I + J
        W_ = list(set(np.arange(self.X.shape[0])) - set(W))
        
        # optimality condition
        m = max(grady[R])
        M = min(grady[S])
        
        flag = False
        if m-M < 1e-2:
            flag = True
            
        return W, W_, Q, flag 
    
    def optimize(self):
        
        start = time.time()
        y = self.y.reshape(-1,1)
        
        old_alpha = np.zeros(self.X.shape[0])
        if self.kernel == "gauss":
            K = self.kernel_gauss(self.X, self.X)
        if self.kernel == "poly":
            K = self.kernel_poly(self.X, self.X)
            
        for i in range(1000):
            
            W, W_, Q, flag = self.get_working_set(self.alpha, K)
            
            if flag:
                print("optimality reached")
                break
            
            # computing alpha
            H = Q
            P = matrix(Q[np.ix_(W,W)])
            q = matrix(old_alpha[W_].T @ Q[np.ix_(W_, W)] - 1)
            G = matrix(np.vstack((-np.eye(len(W)),np.eye(len(W)))))
            h = matrix(np.hstack((np.zeros(len(W)), np.ones(len(W)) * self.C)))
            A = matrix(y[W].reshape(1, -1))
            b = matrix(- y[W_].T @ old_alpha[W_])
            
            solvers.options['show_progress'] = False 
            res = solvers.qp(P, q, G, h, A, b)
            alpha = np.array(res['x']).T
            
            self.alpha[W] = alpha
            old_alpha = copy(self.alpha)
            
            
        end = time.time() - start
        print(end)
        
        # computing b
        K = self.kernel_gauss(self.X, self.X); y = self.y.reshape(-1,1) 
        
        alpha = self.alpha.ravel()
        idx = np.where((alpha > 1e-5) & (alpha < self.C + 1e-5))[0]
        
        wx = (y * self.alpha.reshape(-1,1)).T @ K[:,idx]
        b = y[idx] - wx.T
        
        self.b = np.mean(b)
    
svm = Svm_dcmp(X_train, y_train, gamma = 0.01, C = 2, kernel = "gauss",q = 40)
svm.optimize()
y_pred = svm.predict(X_test)
print(accuracy_score(y_test.reshape(-1,1), y_pred.reshape(-1,1)))

optimality reached
14.313826084136963
0.9025
